In [1]:
import json
import os
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build

In [21]:
SERVICE_ACCOUNT_FILE = 'datadot-10-years-3c9c9647d085.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

SPREADSHEET_ID = '1zA7EzmwMVJzWOrOHG7VlNnnBoLuQGcKD54wgBuivetY'
MAIN_RANGE = 'Respostas ao formulário 1!B1:L1000'
INDS_RANGE = 'IndustryMapper!A1:B100'

In [16]:
def get_service_client():
    # Authenticate using the service account
    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )
    
    # Initialize the Google Sheets API service
    service = build('sheets', 'v4', credentials=credentials)

    return service

In [23]:
def read_spreadsheet(service_client, range):
    result = service_client.spreadsheets().values().get(spreadsheetId=SPREADSHEET_ID, range=range).execute()
    values = result.get('values', [])
    df = pd.DataFrame(values[1:], columns=values[0])
    return df

In [34]:
def map_categories(series, mapper, multivalues=False):
    if (multivalues):
        return series.str.split(", ").apply(lambda arr: [ mapper[val] for val in arr if val != ""])
    else:
        return series.apply(lambda val: mapper[val] if val != "" else None)

In [38]:
# Single-value categories
industryMapper = {
    "Serviços e Tecnologia": "servicos-e-tecnologia",
    "Automotivo": "automotivo",
    "Varejo": "varejo",
    "Educação": "educacao",
    "Financeiro": "financeiro",
    "ONGs e Orgs sociais e ambientais": "ongs-orgs",
    "Consultorias": "consultorias",
    "Imobiliário": "imobiliario",
    "Comunicação e publicidade": "comunicacao",
    "Jornalismo e editorial": "jornalismo",
    "Pesquisa e tendências": "pesquisa"
}

channelMapper = {
    "midia digital": "digital",
    "midia impressa": "impressa",
    "consultoria_servico": "consultoria"
}


# Multiple-values categories

goalMapper = {
    "educacional": "educacional",
    "informacional": "informacional",
    "jornalistico editorial": "jornalistico-editorial",
    "impacto positivo": "impacto-positivo",
    "institucional": "institucional"
    
}

productMapper = {
    "site\x03editorial": "site-editorial",
    "site editorial": "site-editorial",
    "outras\x03interfaces digitais": "outras-interfaces",
    "outras 3interfaces digitais": "outras-interfaces",
    "relatorios": "relatorios",
    "apresentacao": "apresentacao",
    "infografico": "infografico",
    "publicacao": "publicacao",
    "site\x03institucional": "site-institucional",
    "site institucional": "site-institucional",
    "video": "video"
}

designMapper = {
    "user interface": "user-interface",
    'datavis': "datavis",
    'ilustracao': "ilustracao",
    'design de servicos': "design-de-servicos",
    'editorial': "editorial",
    'infografia': "infografia",
    'motion graphics': "motion-graphics"
}

In [24]:
service = get_service_client()

In [25]:
main = read_spreadsheet(service, MAIN_RANGE)
inds = read_spreadsheet(service, INDS_RANGE)

In [43]:
df.industry.unique()

array(['Comunicação e publicidade', 'Pesquisa e tendências',
       'Consultorias', 'Serviços e Tecnologia', 'Educação',
       'ONGs e Orgs sociais e ambientais', 'Imobiliário',
       'Jornalismo e editorial', 'Varejo', 'Financeiro', 'Automotivo', '',
       'Saúde', 'Bens de consumo'], dtype=object)

In [42]:
df.industry.value_counts()

industry
Comunicação e publicidade           100
Financeiro                           78
ONGs e Orgs sociais e ambientais     49
Serviços e Tecnologia                38
Jornalismo e editorial               36
Educação                             26
Consultorias                         20
Pesquisa e tendências                15
Bens de consumo                       7
Automotivo                            6
                                      3
Imobiliário                           2
Varejo                                1
Saúde                                 1
Name: count, dtype: int64

In [39]:
main_cl = main.rename(columns={"product_type (selecionar apenas 1)": "product_type"})

df = main_cl.merge(inds, on="client_name", suffixes=("", "_clean"), how="outer")
df["client_industry"] = df.client_industry_clean.fillna(df.client_industry)
df.drop(columns=["client_industry_clean"], inplace=True)

df.drop(columns=[
    "Carimbo de data/hora", 
    "client_industry_outros", 
    "tem mais de uma entrega"
], inplace=True, errors='ignore')

df.rename(columns={
    "complexity_level": "complexity",
    "assignment_name": "description",
    "tipo de mídia": "channel",
    "product_type": "product",
    "design_category": "design",
    "client_industry": "industry",
    "client_name": "client",
    "project_name": "project"
}, inplace=True)

df["goal"] = map_categories(df["goal"], goalMapper, True)
df["product"] = map_categories(df["product"], productMapper, True)
df["design"] = map_categories(df["design"], designMapper, True)
df["industry"] = map_categories(df["industry"], industryMapper)
df["channel"] = map_categories(df["channel"], channelMapper)

order = [
    "client",
    "industry",
    
    "project",
    "description",
    "dt_start",
    
    "channel", 
    "product",
    "design",
    "goal",

    "complexity"
]

df = df[order]

df.head(3)

KeyError: 'Saúde'

In [31]:
df.head()

,client_name,client_industry,dt_start,project_name,goal,complexity_level,assignment_name,design_category,product_type,tem mais de uma entrega,tipo de mídia
0,Baboom Filmes,Comunicação e publicidade,15/01/2014,Ijen,informacional,2,Infográficos animados para auxiliar na narrati...,"ilustracao, motion graphics, infografia",video,não,midia digital
1,Baboom Filmes,Comunicação e publicidade,20/06/2014,"CPFL - Vídeo 1, energia sustentável","educacional, informacional, institucional",2,Design de linguagem visual e motiongraphics pa...,"ilustracao, motion graphics, infografia",video,sim,midia digital
2,Baboom Filmes,Comunicação e publicidade,01/07/2014,"CPFL - Vídeo 2, energia sustentável","educacional, informacional, institucional",2,"Projeto de linguagem visual, recursos gráficos...","ilustracao, motion graphics, infografia",video,sim,midia digital
3,Baboom Filmes,Comunicação e publicidade,05/01/2015,Infográficos Repórter Record,"informacional, jornalistico editorial",1,Infografias animadas para programa jornalístico,"ilustracao, motion graphics, infografia",video,não,midia digital
4,Project Hub,Comunicação e publicidade,01/05/2014,Youtube - Rio Content Market,"informacional, institucional",3,Aplicativo para realização de pesquisa de opin...,user interface,outrasinterfaces digitais,sim,midia digital


In [33]:
df.to_csv('../../data/formatted.csv', sep=',', index=False)